In [17]:
import pymongo
import pandas as pd
pd.set_option('float_format', '{:f}'.format)
import os
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [3]:
config = {
    'mongo_uri': "mongo-uri"
}

In [5]:
client = pymongo.MongoClient(config['mongo_uri'])
db = client.myLOLDB
collection = db['matchups']
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [34]:
data_list = []
for matchup in collection.find():
    try:
        data = {
            'match_id': matchup['p_match_id'],
            'champion1': matchup['data'][0]['championName'].lower(),
            'champion2': matchup['data'][1]['championName'].lower(),
            'win': 0 if matchup['data'][0]['win'] else 1,
        }
    except:
        print(matchup)

    data_list.append(data)

df = pd.DataFrame(data_list)

{'_id': ObjectId('6647aaf120da708a09b11458'), 'p_match_id': 'BR1_2938548776_', 'data': [{'championName': 'Maokai', 'goldEarned': 7287, 'totalMinionsKilled': 29, 'win': True, 'kills': 1, 'assists': 10, 'deaths': 2, 'visionScore': 54, 'puuid': 'Gmk0B-yPa3CViKNBo2-FbStWqihSAqOCG0OswNY3MYelhknv336NaiVW1BNwUnSwt3xVDDmujXLeTQ', 'totalDamageDealtToChampions': 6719, 'summonerName': 'personal sex'}], 'gameVersion': '14.10.584.5961'}
{'_id': ObjectId('6647aaf220da708a09b11459'), 'p_match_id': 'BR1_2938548776_UTILITY', 'data': [{'championName': 'Milio', 'goldEarned': 7567, 'totalMinionsKilled': 8, 'win': False, 'kills': 0, 'assists': 12, 'deaths': 7, 'visionScore': 42, 'puuid': 'zkmvXg3JNzBHcAaiWnEL0mpYGi_Yi-UTUwUJYIButsZKqP6-aN0P35qfo3-C32_-4CKIJYk51pxiCg', 'totalDamageDealtToChampions': 4264, 'summonerName': 'ZeroQWE'}], 'gameVersion': '14.10.584.5961'}
{'_id': ObjectId('6647ad5e20da708a09b117f8'), 'p_match_id': 'EUN1_3597247732_JUNGLE', 'data': [{'championName': 'Graves', 'goldEarned': 937, 't

In [35]:
df.head(5)

,match_id,champion1,champion2,win
0,BR1_2938902060_TOP,chogath,renekton,0
1,BR1_2938902060_JUNGLE,kindred,viego,0
2,BR1_2938902060_MIDDLE,zoe,ahri,0
3,BR1_2938902060_BOTTOM,caitlyn,ashe,0
4,BR1_2938902060_UTILITY,varus,rumble,0


In [36]:
from sklearn.model_selection import train_test_split

train_dataset, test_dataset = train_test_split(df, test_size=0.2)

# We want to predict the 'win' variable.
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('win') # returns column 'win'
test_labels = test_features.pop('win') # returns column 'win'

In [28]:
import requests

response = requests.get('https://ddragon.leagueoflegends.com/cdn/14.10.1/data/en_US/champion.json')

champion_list = [name.lower() for name in list(response.json()['data'].keys())]

In [37]:
le = LabelEncoder()

le = le.fit(champion_list)

train_features = train_features.drop('match_id', axis=1).apply(lambda x: le.transform(x))
test_features = test_features.drop('match_id', axis=1).apply(lambda x: le.transform(x))

In [39]:
# Normalization
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)
test_features = scaler.transform(test_features)

In [40]:
logreg = LogisticRegression()
logreg.fit(train_features, train_labels)
print('Accuracy of Logistic regression classifier on training set: {:.2f}'
     .format(logreg.score(train_features, train_labels)))
print('Accuracy of Logistic regression classifier on test set: {:.2f}'
     .format(logreg.score(test_features, test_labels)))

Accuracy of Logistic regression classifier on training set: 0.58
Accuracy of Logistic regression classifier on test set: 0.58


In [45]:
new_data = {
    'champ1': ['xayah', 'karma', 'xerath', 'gragas', 'chogath'],
    'champ2': ['tristana', 'lulu', 'syndra', 'sejuani', 'gnar']
}
new_df = pd.DataFrame(new_data)

In [46]:
new_df = new_df.apply(lambda x: le.transform(x))

new_df.tail(5)

,champ1,champ2
0,153,135
1,55,73
2,154,128
3,38,112
4,21,37


In [47]:
result = logreg.predict(new_df)

def find_winner(lst):
    return max(set(lst), key=lst.count)

winner_prediction = find_winner(result.tolist())

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [49]:
inverse_prediction = new_df.apply(lambda x: le.inverse_transform(x)) # we apply inverse transform

if winner_prediction == 1:
    print('Predicted winner is team 1: \n{}'.format(str(inverse_prediction['champ1'])))
else:
     print('Predicted winner is team 2: \n{}'.format(str(inverse_prediction['champ2'])))

Predicted winner is team 1: 
0      xayah
1      karma
2     xerath
3     gragas
4    chogath
Name: champ1, dtype: object
